# Example: Let's Explore Binomial Lattice Model Trade Exit Rules
In this example, we will test the binomial lattice model trade exit rules we developed in lecture.
In particular, we will use a binomial lattice model to compute the cumulative probability of achieving at least a target fractional return (ROI) of $r_{\star}$ (or alternatively, a target fractional loss of $-r_{\star}$) over a holding period of $T=i\Delta{t}$.

> __Leaning Objectives__
> 
> Fill me in later!

This sounds cool, so let's get started!
___